In [ ]:
# Google Colab environment
import os
import urllib.request
import zipfile
import tarfile
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

batch_size = 32

url = "http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz"
annotations_url = "http://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat"
split_url = "http://www.robots.ox.ac.uk/~vgg/data/flowers/102/setid.mat"

# Directories for the dataset and annotations
data_dir = "./flowers102"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Download the dataset
urllib.request.urlretrieve(url, os.path.join(data_dir, "102flowers.tgz"))
urllib.request.urlretrieve(annotations_url, os.path.join(data_dir, "imagelabels.mat"))
urllib.request.urlretrieve(split_url, os.path.join(data_dir, "setid.mat"))

# Extract the dataset
with tarfile.open(os.path.join(data_dir, "102flowers.tgz"), "r:gz") as tar:
    tar.extractall(path=data_dir)

import scipy.io
import shutil
from sklearn.model_selection import train_test_split

# Load the labels and splits
labels = scipy.io.loadmat(os.path.join(data_dir, "imagelabels.mat"))["labels"][0]
setid = scipy.io.loadmat(os.path.join(data_dir, "setid.mat"))
train_ids = setid["trnid"][0]
val_ids = setid["valid"][0]
test_ids = setid["tstid"][0]

def create_dataset_split(ids, split_name):
    split_dir = os.path.join(data_dir, split_name)
    if not os.path.exists(split_dir):
        os.makedirs(split_dir)

    for idx in ids:
        label = labels[idx - 1]
        src_path = os.path.join(data_dir, "jpg", f"image_{idx:05d}.jpg")
        dest_dir = os.path.join(split_dir, str(label))
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)
        shutil.move(src_path, os.path.join(dest_dir, f"image_{idx:05d}.jpg"))

# Create train, validation, and test splits
create_dataset_split(train_ids, "train")
create_dataset_split(val_ids, "val")
create_dataset_split(test_ids, "test")

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "test": transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Create datasets
image_datasets = {
    "train": datasets.ImageFolder(os.path.join(data_dir, "train"), data_transforms["train"]),
    "val": datasets.ImageFolder(os.path.join(data_dir, "val"), data_transforms["val"]),
    "test": datasets.ImageFolder(os.path.join(data_dir, "test"), data_transforms["test"])
}

# Create dataloaders
dataloaders = {
    "train": DataLoader(image_datasets["train"], batch_size=batch_size, shuffle=True, num_workers=2),
    "val": DataLoader(image_datasets["val"], batch_size=batch_size, shuffle=False, num_workers=2),
    "test": DataLoader(image_datasets["test"], batch_size=batch_size, shuffle=False, num_workers=2)
}

# for inputs, labels in dataloaders['train']:
#     print(inputs.shape, labels.shape)
#     break

In [ ]:
"""import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Specify dataset directory
data_dir = 'path/to/your/dataset'

# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load dataset
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])

# Split dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
"""
train_dataset = image_datasets['train']
val_dataset = image_datasets['val']
test_dataset = image_datasets['test']
train_loader = dataloaders['train']
val_loader = dataloaders['val']
test_loader = dataloaders['test']

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Define the Attention Model
class AttentionModel(nn.Module):
    def __init__(self, num_classes):
        super(AttentionModel, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Identity()  # Remove the fully connected layer

        self.attention_layer = nn.Sequential(
            nn.Conv2d(2048, 512, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(512, 1, kernel_size=1)
        )

        self.classifier = nn.Linear(2048, num_classes)

    def forward(self, x):
        features = self.resnet(x)

        # Reshape features to keep spatial dimensions
        features = features.view(features.size(0), 2048, 1, 1)

        attention = self.attention_layer(features)
        attention = torch.sigmoid(attention)
        attention_features = features * attention

        pooled_features = nn.AdaptiveAvgPool2d((1, 1))(attention_features)
        pooled_features = pooled_features.view(pooled_features.size(0), -1)
        output = self.classifier(pooled_features)

        return output

# Initialize the model
num_classes = len(train_dataset.classes)  # Adjust to match the number of classes in your dataset
model = AttentionModel(num_classes=num_classes).to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 185MB/s]


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Define training parameters
num_epochs = 16


In [ ]:
import copy

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
            dataloader = train_loader
        else:
            model.eval()  # Set model to evaluate mode
            dataloader = val_loader

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                # Backward pass and optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Deep copy the model
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

# Load best model weights
model.load_state_dict(best_model_wts)


Epoch 0/15
----------
train Loss: 4.6203 Acc: 0.0167
val Loss: 4.5682 Acc: 0.0363
Epoch 1/15
----------
train Loss: 4.5325 Acc: 0.0735
val Loss: 4.4263 Acc: 0.1314
Epoch 2/15
----------
train Loss: 4.3434 Acc: 0.1755
val Loss: 4.0746 Acc: 0.3304
Epoch 3/15
----------
train Loss: 3.9571 Acc: 0.3706
val Loss: 3.5376 Acc: 0.4804
Epoch 4/15
----------
train Loss: 3.4958 Acc: 0.5147
val Loss: 2.9699 Acc: 0.5863
Epoch 5/15
----------
train Loss: 3.0182 Acc: 0.6029
val Loss: 2.5245 Acc: 0.6284
Epoch 6/15
----------
train Loss: 2.6094 Acc: 0.6686
val Loss: 2.1372 Acc: 0.6833
Epoch 7/15
----------
train Loss: 2.2257 Acc: 0.7255
val Loss: 1.7924 Acc: 0.7314
Epoch 8/15
----------
train Loss: 1.9265 Acc: 0.7559
val Loss: 1.5523 Acc: 0.7735
Epoch 9/15
----------
train Loss: 1.6587 Acc: 0.8059
val Loss: 1.3452 Acc: 0.8010
Epoch 10/15
----------
train Loss: 1.4116 Acc: 0.8294
val Loss: 1.1764 Acc: 0.8265
Epoch 11/15
----------
train Loss: 1.2440 Acc: 0.8422
val Loss: 1.0398 Acc: 0.8392
Epoch 12/15
--

<All keys matched successfully>

In [ ]:
# Evaluate the model on the test set
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

model.eval()
running_corrects = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        running_corrects += torch.sum(preds == labels.data)

test_acc = running_corrects.double() / len(test_loader.dataset)
print(f'Test Acc: {test_acc:.4f}')


Test Acc: 0.8486
